In [ ]:
pip install rdflib

In [ ]:
import pandas as pd;
import numpy as np; 

from rdflib import Graph, Literal, RDF, URIRef, Namespace , BNode
from rdflib.namespace import FOAF , XSD, DC, FOAF, SKOS, RDF, RDFS
import urllib.parse 
import hashlib
import uuid

In [ ]:
pd.set_option('display.max_colwidth', 0)

## CONSTANTS

In [ ]:
ROAD_SIGN_DATA_SET = 'https://raw.githubusercontent.com/lblod/app-mow-registry/main/config/migrations/20190821152302-verkeersborden.ttl'

traffic_light_concept = Namespace('http://data.vlaanderen.be/id/concept/Verkeerslichtconcept/')
road_marking_concept = Namespace('http://data.vlaanderen.be/id/concept/Wegmarkeringsconcept/')

## Helper functions

In [ ]:
def concept_uri(base_uri, input):
  m = hashlib.md5()
  m.update(input.encode('utf-8'))

  return (URIRef(base_uri + m.hexdigest()), m.hexdigest())

def add_literal(g, subject, predicate, object_value, datatype=None):
  if object_value != str(np.nan):
    if datatype == None:
      g.add((subject, predicate, Literal(object_value, lang='nl')))
    else:
      g.add((subject, predicate, Literal(object_value, datatype=datatype)))

verkeersborden = Graph().parse(ROAD_SIGN_DATA_SET, format='turtle')

def find_road_sign(code):
  r = verkeersborden.query("SELECT ?s WHERE {{?s <http://www.w3.org/2004/02/skos/core#prefLabel> \"{0}\"}}".format(code))
  return next(x for x in r)[0]

## Harvest HTML pages

In [ ]:
dfl = pd.read_html('/content/drive/MyDrive/colab_data/LijstVerkeersSignalisatieLichtenIcon.html')[0];  

In [ ]:
dfm = pd.read_html('/content/drive/MyDrive/colab_data/LijstVerkeersSignalisatieWegIcon.html')[0];  

In [ ]:
dfm['code'] = 'WM' + dfm['Plaatsing'].str.extract(r'([67]\d[.\d]*)')[0].str.rstrip('.')
#FIXME : hack to avoid non unique code. Two definitions/variations under same article in wegcode 
dfm.loc[dfm['opschrift'] == 'Onderbroken trekken dichter bij elkaar','code'] = 'WM72.3.b'

In [ ]:
dfl['code'] = 'VL' + dfl['Plaatsing'].str.replace(' ','').str.extract(r'([67]\d[.\d]*(bis)?(ter)?)')[0].str.rstrip('.')


In [ ]:
dfl['related'] = dfl['Combinatie'] \
  .str.replace('LiFP','LiFV, LiFO')\
  .str.replace('LI', 'Li') \
  .str.replace('VO', 'Vo') \
  .str.findall(r'(Li[\d\w]*)')
dfl['related'] = dfl['related'].apply(lambda d: d if isinstance(d, list) else [])

In [ ]:
dfm['related'] = dfm[dfm.Combinatie.notnull()]['Combinatie'].apply(find_road_sign)

In [ ]:
dfl['related'] = dfl['related'].apply(lambda d: d if isinstance(d, list) else [])

In [ ]:
dfm.code.fillna(dfm.Benaming, inplace=True)


In [ ]:
dfl['uri'], dfl['uuid'] = zip(*dfl['code'].apply(lambda x: concept_uri(traffic_light_concept, x)))
dfm['uri'], dfm['uuid'] = zip(*dfm['code'].apply(lambda x: concept_uri(road_marking_concept, x)))

# Graph

VerkeersbordConcept basis
```
<http://data.vlaanderen.be/id/concept/Verkeersbordconcept/00939dd22288b0ef321908552d49f97c6e77e53de6e14b303f32c9a499839557> a <https://data.vlaanderen.be/ns/mobiliteit#Verkeersbordconcept>;
   <http://mu.semte.ch/vocabularies/core/uuid> "00939dd22288b0ef321908552d49f97c6e77e53de6e14b303f32c9a499839557";
   <http://www.w3.org/2003/06/sw-vocab-status/ns#termStatus> <http://mow.lblod.info/VerkeersbordconceptStatus/de811976b13d96ba62aec77411a88d0ad9afc573221f2c8dee979cd8106ea77b>;
   <http://www.w3.org/2004/02/skos/core#definition> "Smalle doorgang. Voorrang ten opzichte van de bestuurders die uit de tegenovergestelde richting komen.";
   <http://www.w3.org/2004/02/skos/core#inScheme> <http://data.vlaanderen.be/id/conceptscheme/Verkeersbordconcept>;
   <http://www.w3.org/2004/02/skos/core#prefLabel> "B21";
   <http://www.w3.org/2004/02/skos/core#scopeNote> "Smalle doorgang. Voorrang ten opzichte van de bestuurders die uit de tegenovergestelde richting komen.";
   <http://www.w3.org/2004/02/skos/core#topConceptOf> <http://data.vlaanderen.be/id/conceptscheme/Verkeersbordconcept>;
   <http://www.w3.org/ns/org#classification> <http://data.vlaanderen.be/id/concept/Verkeersbordcatergorie/737da5751bc7f311398a834f34df310dd95255a0b62afa2db2882c72d54b47d2>;
   <https://data.vlaanderen.be/ns/mobiliteit#grafischeWeergave> <http://mobiliteit.vo.data.gift/images/efc5dcdd73633d3a864dea5e8d4ed57da8a6bb5bab78883d17302899ccaf0d06> .
```



In [ ]:
g = Graph()
f = Graph()

In [ ]:
for index, row in dfl.iterrows():
  s = row['uri']
  g.add((s, RDF.type, URIRef('https://data.vlaanderen.be/ns/mobiliteit#Verkeerslichtconcept')))
  g.add((s, URIRef('http://mu.semte.ch/vocabularies/core/uuid'), Literal(row['uuid'])))
  #TODO
  g.add((s, URIRef('http://www.w3.org/2003/06/sw-vocab-status/ns#termStatus'), URIRef('http://mow.lblod.info/VerkeersbordconceptStatus/de811976b13d96ba62aec77411a88d0ad9afc573221f2c8dee979cd8106ea77b')))
  add_literal(g, s, SKOS.definition, row['opschrift'])
  add_literal(g, s, SKOS.scopeNote, row['Betekenis'])
  g.add((s, SKOS.inScheme, URIRef('http://data.vlaanderen.be/id/conceptscheme/Verkeerslichtconcept')))
  g.add((s, SKOS.prefLabel, Literal(row['code'])))
  g.add((s, SKOS.topConceptOf, URIRef('http://data.vlaanderen.be/id/conceptscheme/Verkeerslichtconcept')))
  #TODO
  g.add((s, URIRef('http://www.w3.org/ns/org#classification'), URIRef('http://data.vlaanderen.be/id/concept/Verkeersbordcatergorie/737da5751bc7f311398a834f34df310dd95255a0b62afa2db2882c72d54b47d2')))
  #TODO
  g.add((s, URIRef('https://data.vlaanderen.be/ns/mobiliteit#grafischeWeergave'), URIRef('http://localhost/assets/' + row['Icoon'])))
  for value in row['related']:
    u = dfl[dfl['Benaming'] == value]['uri']
    if not u.empty:
      g.add((s, URIRef('http://data.lblod.info/vocabularies/mobiliteit/heeftGerelateerdVerkeerslichtconcept'), URIRef(u.iloc[0])))
    else:
      print("Not found:" + value)

In [ ]:
print(g.serialize(format='turtle'))

@prefix ns1: <http://data.lblod.info/vocabularies/mobiliteit/> .
@prefix ns2: <http://www.w3.org/2004/02/skos/core#> .
@prefix ns3: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix ns4: <https://data.vlaanderen.be/ns/mobiliteit#> .
@prefix ns5: <http://mu.semte.ch/vocabularies/core/> .
@prefix ns6: <http://www.w3.org/ns/org#> .

<http://data.vlaanderen.be/id/concept/Verkeerslichtconcept/41b57854c253fb185a69a9168e94c2e9> a ns4:Verkeerslichtconcept ;
    ns5:uuid "41b57854c253fb185a69a9168e94c2e9" ;
    ns3:termStatus <http://mow.lblod.info/VerkeersbordconceptStatus/de811976b13d96ba62aec77411a88d0ad9afc573221f2c8dee979cd8106ea77b> ;
    ns2:definition "Oranje knipperlicht"@nl ;
    ns2:inScheme <http://data.vlaanderen.be/id/conceptscheme/Verkeerslichtconcept> ;
    ns2:prefLabel "VL64" ;
    ns2:scopeNote "een oranjegeel knipperlicht betekent dat het verkeerslicht met dubbele voorzichtigheid mag voorbij gereden worden; het wijzigt de voorrangsregeling niet."@nl ;
    ns2:topConc

In [ ]:
g.serialize('verkeerslichten.ttl', format='turtle')

In [ ]:
for index, row in dfm[dfm['TypeBord'] == 'Wegmarkering'].iterrows():
  s = row['uri']
  f.add((s, RDF.type, URIRef('https://data.vlaanderen.be/ns/mobiliteit#Wegmarkeringconcept')))
  f.add((s, URIRef('http://mu.semte.ch/vocabularies/core/uuid'), Literal(row['uuid'])))
  #TODO
  f.add((s, URIRef('http://www.w3.org/2003/06/sw-vocab-status/ns#termStatus'), URIRef('http://mow.lblod.info/WegmarkeringconceptStatus/de811976b13d96ba62aec77411a88d0ad9afc573221f2c8dee979cd8106ea77b')))
  add_literal(f, s, SKOS.definition, row['opschrift'])
  add_literal(f, s, SKOS.scopeNote, row['Betekenis'])
  f.add((s, SKOS.inScheme, URIRef('http://data.vlaanderen.be/id/conceptscheme/Wegmarkeringconcept')))
  f.add((s, SKOS.prefLabel, Literal(row['code'])))
  f.add((s, SKOS.topConceptOf, URIRef('http://data.vlaanderen.be/id/conceptscheme/Wegmarkeringconcept')))
  #TODO
  f.add((s, URIRef('http://www.w3.org/ns/org#classification'), URIRef('http://data.vlaanderen.be/id/concept/Wegmarkeringcatergorie/737da5751bc7f311398a834f34df310dd95255a0b62afa2db2882c72d54b47d2')))
  #TODO
  f.add((s, URIRef('https://data.vlaanderen.be/ns/mobiliteit#grafischeWeergave'), URIRef('http://localhost/assets/' + row['Icoon'])))
  if isinstance(row['related'], URIRef) :
    f.add((s, URIRef('http://data.lblod.info/vocabularies/mobiliteit/heeftGerelateerdVerkeersbordconcept'), row['related']))
  

In [ ]:
print(f.serialize(format='turtle'))

@prefix ns1: <http://www.w3.org/2004/02/skos/core#> .
@prefix ns2: <http://mu.semte.ch/vocabularies/core/> .
@prefix ns3: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix ns4: <http://data.lblod.info/vocabularies/mobiliteit/> .
@prefix ns5: <http://www.w3.org/ns/org#> .
@prefix ns6: <https://data.vlaanderen.be/ns/mobiliteit#> .

<http://data.vlaanderen.be/id/concept/Wegmarkeringsconcept/053e2451065c5bd0946740fe662c2612> a ns6:Wegmarkeringconcept ;
    ns4:heeftGerelateerdVerkeersbordconcept <http://data.vlaanderen.be/id/concept/Verkeersbordconcept/e076d4553d8995d0739ad74dc30dbb5cf4095b6c0a9e9a68503e7a7da03f3b9e> ;
    ns2:uuid "053e2451065c5bd0946740fe662c2612" ;
    ns3:termStatus <http://mow.lblod.info/WegmarkeringconceptStatus/de811976b13d96ba62aec77411a88d0ad9afc573221f2c8dee979cd8106ea77b> ;
    ns1:definition "Haaietanden"@nl ;
    ns1:inScheme <http://data.vlaanderen.be/id/conceptscheme/Wegmarkeringconcept> ;
    ns1:prefLabel "WM76.2" ;
    ns1:scopeNote "Een dwarsstre

In [ ]:
f.serialize('wegmarkeringen.ttl', format='turtle')